In [1]:
import torch
from tqdm import tqdm
from transformers import (
    AutoConfig,
    AutoModelForMultipleChoice,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
import numpy as np
# np.argmax([1,2,3,4])
def softmax(x, axis=None):
    x = np.array(x)
    x = x - x.max(axis=axis, keepdims=True)
    y = np.exp(x)
    rst = y / y.sum(axis=axis, keepdims=True)
    return rst.tolist()
softmax([1,2])
tokenizer = AutoTokenizer.from_pretrained('ethanyt/guwenbert-base')

In [2]:
import json
with open('data/valid.jsonl','r') as f:
    valid_json = f.readlines()
    valid_json = [json.loads(e) for e in valid_json]

### co_num

In [12]:
def multi_argmin(l):
    min_l = min(l)
    return [i for i in range(len(l)) if l[i] == min_l]
def multi_argmax(l):
    max_l = max(l)
    return [i for i in range(len(l)) if l[i] == max_l]
def choice_cooccurrence(choices):
    num = len(choices)
    choices_return = []
    scores = [0] * num
    choice_tokens = [[] for _ in range(num)]
    for i, choice in enumerate(choices):
        choices_return.append(choice)
        choice_tokens[i] = choice
    for i in range(num):
        for j in range(num):
            if i == j:
                continue
            for choice_tokens_id in range(len(choice_tokens[i])):
                if choice_tokens[i][choice_tokens_id] == choice_tokens[i][choice_tokens_id]:
                    scores[i] += 1
    min_choice = multi_argmin(scores)
    return scores
    return min_choice
    if len(min_choice) > 3:
        return choices_return
    for i in min_choice:
        choices_return[i] = ''
    return choices_return
def choice_cooccurrence_no_position(choices):
    num = len(choices)
    choices_return = []
    scores = [0] * num
    choice_tokens = [[] for _ in range(num)]
    for i, choice in enumerate(choices):
        choices_return.append(choice)
        for c in choice:
            if c in choice_tokens[i]:
                continue
            choice_tokens[i].append(c)
    for i in range(num):
        for j in range(num):
            if i == j:
                continue
            for c in choice_tokens[i]:
                if c in choice_tokens[j]:
                    scores[i] += 1
    min_choice = multi_argmin(scores)
    return min_choice
    if len(min_choice) > 3:
        return choices_return
    for i in min_choice:
        choices_return[i] = ''
    return choices_return

In [13]:
valid_json[0]['choices'],choice_cooccurrence_no_position(valid_json[0]['choices']),choice_cooccurrence(valid_json[0]['choices'])

(['渔灯灭复明', '残灯灭又然', '残灯暗复明', '残灯灭又明'], [0, 1, 2], [15, 15, 15, 15])

In [86]:
def calculate_translation_with_choice(translations, choice):
    rst = 0
    for c in choice:
        if c in translations:
            rst += 1
    return rst
def calculate_choice_in_choices(choice,choices):
    rst = 0
    for c in choice:
        for other_choice in choices:
            if choice == other_choice:
                continue
            if c in other_choice:
                rst += 1
    return rst
def calculate_choice_in_position_choices(choice,choices):
    rst = 0
    for c_id in range(len(choice)):
        c = choice[c_id]
        for other_choice in choices:
            if choice == other_choice:
                continue
            other_c = other_choice[c_id]
            if c == other_c:
                rst += 1
    return rst
    
predict_valid_json = []
for i in range(len(valid_json)):
    choices = valid_json[i]['choices']
#     choices = choice_cooccurrence(choices)
    this_scores = []
    for choice_id in range(len(choices)):
        choice_score = calculate_translation_with_choice(valid_json[i]['translation'], choices[choice_id])
        choice_score += 0.5 * calculate_choice_in_choices(choices[choice_id], choices)
#         choice_score += calculate_choice_in_position_choices(choices[choice_id], choices)
#         choice_score = sum(choice_score)
        this_scores.append(choice_score)
    this_dict = {k:valid_json[i][k] for k in valid_json[i]}
    this_scores = softmax(this_scores)
    this_dict['scores'] = this_scores
    this_dict['predict'] = np.argmax(this_scores)
    this_dict['no_predict'] = multi_argmin(this_scores)
    this_dict['true'] = (this_dict['predict'] == this_dict['answer'])
    predict_valid_json.append(this_dict)
np.sum([int(e['true']) for e in predict_valid_json]) / len(predict_valid_json)

0.8768382352941176

In [87]:
co_num_predict = predict_valid_json

In [88]:
sum([e['answer'] in e['no_predict'] for e in co_num_predict]) / len(predict_valid_json)

0.020955882352941175

In [89]:
co_num_predict[-1]

{'translation': '便被选进了宫中。',
 'choices': ['王孙归去好', '归去越王家', '归去葛洪家', '归去帝王家'],
 'answer': 1,
 'scores': [0.13447071068499755,
  0.36552928931500245,
  0.13447071068499755,
  0.36552928931500245],
 'predict': 1,
 'no_predict': [0, 2],
 'true': True}

### ner


In [90]:
import pickle
with open('predict/final/eval/ner.pkl','rb') as f:
    rst = pickle.load(f)
all_scores = {}
for batch_idx in tqdm(range(len(rst['test_dataset']))):
    origin_id = rst['test_dataset'][batch_idx]['origin_idx']
    if origin_id not in all_scores:
        all_scores[origin_id] = []
    predicts = softmax(rst['test_result'].predictions[batch_idx],axis=-1)
    labels = rst['test_result'].label_ids[batch_idx]
    all_scores[origin_id].append(np.array(predicts)[np.where(labels!=-100)[0].tolist()][:,1].tolist())
def multiple_list(l):
    rst = 1
    for e in l:
        rst *= e
    return rst

predict_valid_json = []
for i in range(len(valid_json)):
    choices = valid_json[i]['choices']
    this_scores = []
    for choice_id in range(len(choices)):
        choice_score = all_scores[i][choice_id]
        choice_score = multiple_list(choice_score)
#         choice_score = sum(choice_score)
        this_scores.append(choice_score)
    this_dict = {k:valid_json[i][k] for k in valid_json[i]}
    this_scores = np.array(this_scores)
#     this_scores[co_num_predict[i]['no_predict']] += -100
    this_scores = this_scores.tolist()
    this_dict['scores'] = softmax(this_scores)
    this_dict['predict'] = np.argmax(this_scores)
    this_dict['true'] = (this_dict['predict'] == this_dict['answer'])
    predict_valid_json.append(this_dict)

100%|████████████████████████████████████████████████████████████████████████| 10880/10880 [00:08<00:00, 1349.83it/s]


In [91]:
np.sum([int(e['true']) for e in predict_valid_json]) / len(predict_valid_json)

0.8319852941176471

save/ner/guwen/checkpoint-15000/pytorch_model.bin

0.8224264705882353

18000
0.8220588235294117

12000
0.8319852941176471

9000
0.81875

6000
0.8183823529411764

3000
0.8338235294117647

21000
0.830514705882353

24000
0.8158088235294118

27000
0.8242647058823529

30000
0.8202205882352941

33000
0.8213235294117647

In [92]:
ner_predict = predict_valid_json

### binary

In [93]:
import pickle
with open('predict/final/eval/binary.pkl','rb') as f:
    rst = pickle.load(f)
all_scores = {}
for i in tqdm(range(len(rst['test_dataset']))):
    idx = rst['test_dataset'][i]['origin_idx']
    input_str = rst['test_dataset'][i]['input_ids']
    input_str = tokenizer.decode(input_str)
    input_str = input_str.split('[SEP]')
    if idx not in all_scores:
        all_scores[idx] = {}
    all_scores[idx][input_str[1].replace(' ','')] = softmax(rst['test_result'].predictions[i].tolist())

predict_valid_json = []
for i in range(len(valid_json)):
    choices = valid_json[i]['choices']
    this_scores = []
    for choice in choices:
        choice_score = 0
        for k in all_scores[i]:
            if k in choice:
                choice_score += all_scores[i][k][1]
        this_scores.append(choice_score)
    this_dict = {k:valid_json[i][k] for k in valid_json[i]}
    this_scores = np.array(this_scores)
#     this_scores[co_num_predict[i]['no_predict']] += -100
    this_scores = this_scores.tolist()
    this_dict['scores'] = softmax(this_scores)
    this_dict['predict'] = np.argmax(this_scores)
    this_dict['true'] = (this_dict['predict'] == this_dict['answer'])
    predict_valid_json.append(this_dict)

np.sum([int(e['true']) for e in predict_valid_json]) / len(predict_valid_json)

100%|████████████████████████████████████████████████████████████████████████| 24797/24797 [00:09<00:00, 2538.26it/s]


0.8268382352941176

0.8172794117647059

3000
0.8268382352941176

6000
0.8180147058823529

In [94]:
binary_predict = predict_valid_json

### seq

In [95]:
import pickle
with open('predict/final/eval/seq.pkl','rb') as f:
    rst = pickle.load(f)
all_scores = {}
for i in tqdm(range(len(rst['test_dataset']))):
    idx = i
    all_scores[idx] = softmax(rst['test_result'].predictions[i].tolist())

predict_valid_json = []
for i in range(len(valid_json)):
    choices = valid_json[i]['choices']
    this_scores = all_scores[i]
    this_dict = {k:valid_json[i][k] for k in valid_json[i]}
    this_scores = np.array(this_scores)
#     this_scores[co_num_predict[i]['no_predict']] += -100
    this_scores = this_scores.tolist()
    this_dict['scores'] = softmax(this_scores)
    this_dict['predict'] = np.argmax(this_scores)
    this_dict['true'] = (this_dict['predict'] == this_dict['answer'])
    predict_valid_json.append(this_dict)
np.sum([int(e['true']) for e in predict_valid_json]) / len(predict_valid_json)

100%|█████████████████████████████████████████████████████████████████████████| 2720/2720 [00:00<00:00, 32867.31it/s]


0.8830882352941176

In [96]:
seq_predict = predict_valid_json

### ensemble

In [97]:
oracle_true = [binary_predict[i]['true'] or ner_predict[i]['true'] or seq_predict[i]['true'] or co_num_predict[i]['true'] for i in range(len(ner_predict))]
oracle_true = sum(oracle_true) / len(oracle_true)
print(oracle_true)

0.9808823529411764


In [103]:
w = [1,0.1,1.5,2]
predict_list = [ner_predict,binary_predict,seq_predict,co_num_predict]
ensemble_predict = []
def get_score(w,predict_list):
    ensemble_predict = []
    for i in range(len(predict_list[0])):
        ensemble_predict.append(predict_list[0][i].copy())
        ensemble_predict[-1]['scores'] = [np.array(predict_list[each_ensemble][i]['scores']) * w[each_ensemble] 
                                          for each_ensemble in range(len(predict_list))]
        ensemble_predict[-1]['scores'] = sum(ensemble_predict[-1]['scores']).tolist()
        ensemble_predict[-1]['predict'] = np.argmax(ensemble_predict[-1]['scores'])
        ensemble_predict[-1]['true'] = (ensemble_predict[-1]['predict'] == ensemble_predict[-1]['answer'])
    return np.sum([int(e['true']) for e in ensemble_predict]) / len(ensemble_predict)
get_score(w,predict_list)

0.925

In [31]:
for i in range(100):
    w1 = i * 0.1
    print(w1,' ',get_score([w[0],w[1],w1],predict_list))

0.0   0.875
0.1   0.8919117647058824
0.2   0.8926470588235295
0.30000000000000004   0.8948529411764706
0.4   0.8952205882352942
0.5   0.8944852941176471
0.6000000000000001   0.8944852941176471
0.7000000000000001   0.8955882352941177
0.8   0.8952205882352942
0.9   0.8959558823529412
1.0   0.8952205882352942
1.1   0.8952205882352942
1.2000000000000002   0.8952205882352942
1.3   0.8952205882352942
1.4000000000000001   0.8959558823529412
1.5   0.8966911764705883
1.6   0.8959558823529412
1.7000000000000002   0.8959558823529412
1.8   0.8955882352941177
1.9000000000000001   0.8955882352941177
2.0   0.8955882352941177
2.1   0.8955882352941177
2.2   0.8952205882352942
2.3000000000000003   0.8952205882352942
2.4000000000000004   0.8952205882352942
2.5   0.8948529411764706
2.6   0.8948529411764706
2.7   0.8948529411764706
2.8000000000000003   0.8948529411764706
2.9000000000000004   0.8948529411764706
3.0   0.8948529411764706
3.1   0.8948529411764706
3.2   0.8944852941176471
3.3000000000000003   0